In [17]:
# author: Asmaa ~ 2019
# ----------------------

# import libraries
import os
import zipfile
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from google.colab import files
from keras.preprocessing import image

Using TensorFlow backend.


# Data Preparation

### Loading Training and Validation Images

In [2]:
# load training zip file
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2019-08-09 12:12:46--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c0f::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   193MB/s    in 0.7s    

2019-08-09 12:12:46 (193 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [3]:
# load validation zip file
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2019-08-09 12:12:48--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  --.-KB/s    in 0.05s   

2019-08-09 12:12:48 (235 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



### Extracting .zip Files

In [0]:
# extract zip file
local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

### Pre-processing

In [5]:
# set rescaling factor
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


# Building the Model 

### Setting the Structure of the Model

In [0]:
model = Sequential()

# first convolution with 300x300 input (the size of the images) and 3 color channels
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)))

# 2x2 pooling
model.add(MaxPooling2D(2, 2))

# second convolution
model.add(Conv2D(32, (3,3), activation='relu'))

# pooling
model.add(MaxPooling2D(2, 2))

# fourth convolution
model.add(Conv2D(64, (3,3), activation='relu'))

# pooling
model.add(MaxPooling2D(2, 2))

# Flatten the result to feed it into the first layer of the DNN
model.add(Flatten())

# hidden layer
model.add(Dense(units=512, activation='relu'))

# outpt layer which has only 1 unit since it is a binary classificaiton problem
model.add(Dense(units=1, activation='sigmoid'))


### Getting a Summary about the CNN

In [13]:
# get a summary about the NN
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 78400)            

### Configuring  the Specifications of the Model

In [0]:
# configure the specifications for model training
# lr = learning rate
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])

### Starting Training

In [16]:
history = model.fit_generator( train_generator, steps_per_epoch=8, epochs=15,
      verbose=1, validation_data = validation_generator, validation_steps=8)

Epoch 1/15
8/8 [==============================] - 10s 1s/step - loss: 9.7800 - acc: 0.5239 - val_loss: 0.6737 - val_acc: 0.5000
Epoch 2/15
8/8 [==============================] - 7s 822ms/step - loss: 0.9219 - acc: 0.6385 - val_loss: 0.5387 - val_acc: 0.7695
Epoch 3/15
8/8 [==============================] - 6s 783ms/step - loss: 0.5588 - acc: 0.8109 - val_loss: 2.3344 - val_acc: 0.5391
Epoch 4/15
8/8 [==============================] - 7s 873ms/step - loss: 0.4901 - acc: 0.8008 - val_loss: 1.2319 - val_acc: 0.6914
Epoch 5/15
8/8 [==============================] - 6s 784ms/step - loss: 0.1687 - acc: 0.9611 - val_loss: 1.3701 - val_acc: 0.7852
Epoch 6/15
8/8 [==============================] - 5s 676ms/step - loss: 0.4186 - acc: 0.8824 - val_loss: 0.5520 - val_acc: 0.8477
Epoch 7/15
8/8 [==============================] - 7s 877ms/step - loss: 0.2187 - acc: 0.9004 - val_loss: 1.6658 - val_acc: 0.7695
Epoch 8/15
8/8 [==============================] - 6s 807ms/step - loss: 0.2692 - acc: 0.9255

### Using the Model

In [19]:
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving horse.JPG to horse.JPG
[0.]
horse.JPG is a horse
